# Resumo do código

### Código que gera o ficheiro: 2_Delta
---
O objectivo é receber dados dos ficheiros totais da Delta e devolver um ficheiro com os dados dos produtos e das lojas relevantes para as missões realizadas nos dias em que foram realizadas as missões. 

Devolve um <u>ficheiro que pode entrar no código 2</u> para juntar aos dados dos ninjas.

---
- Inputs

> __Dados da Delta Todos__: Lojas e produtos todos
> - Todas as métricas possíveis de retirar só dos dados Delta


- Outputs

> **2_Delta**:
> - Ficheiro com todas as métricas nas datas correspondentes aos dias em que ocorreram as missões


In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

# Ler ficheiro com tudo e os específicos

- Tudo

In [4]:
%%time
#dfTudo=pd.read_csv('D:\\B&N Dados\\Delta\\Stocks2023\\StocksDelta2023.csv')

dfTudo=pd.read_excel('D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta2.xlsx', sheet_name="Sheet2")

CPU times: total: 375 ms
Wall time: 396 ms


> Passar para datetime

In [10]:
dfTudo['DATA']= pd.to_datetime(dfTudo['DATA'], format='%Y-%m-%d')

# Encontrar os nomes no dataFrame de tudo

In [11]:
dfTudo['STORE_NAME'].sort_values().unique()


#Lojas:
                  #Reposição=Sim
#CNT AMADORA
#CNT BRAGA
#CNT CASCAIS
#CNT COIMBRASHOPPING
#CNT MATOSINHOS
                  #Reposição=Não
#CNT LEIRIA
#CNT SANTAREM
#CNT VIANA
#MDL T. VEDRAS                                              
#CNT PORTIMÃO

#CAFÉ DELTA Q QALIDUS 80CAP
#CAFÉ DELTA Q MYTHIQ 80CAP
#CAFÉ DELTA Q QALIDUS 10CAP
#CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G
#CAFÉ BELLISSIMO INTENSO 200GR
#CAFÉ DELTA RITUAL MU 220G
#CAFÉ SOLÚVEL DELTA FRASCO 200G
#CAFÉ SOLÚVEL DELTA FRASCO 100G
#CEVADA SOLÚVEL DELTA FRASCO 200G

#Produto novo

array(['CBD ALBERGARIA', 'CBD ALCANTARILHA', 'CBD ALCOCHETE',
       'CBD ALFORNELOS', 'CBD ALFRAGIDE', 'CBD ALGÉS', 'CBD ALPENDURADA',
       'CBD ALPIARCA', 'CBD ALVALADE', 'CBD AMIAL', 'CBD AQUA DE ROMA',
       'CBD AREOSA', 'CBD ARMAÇAO DE PERA', 'CBD ARRUDA VINHOS',
       'CBD ASPRELA', 'CBD AV BOAVISTA', 'CBD AVANCA',
       'CBD AVEIRO ESTAÇÃO', 'CBD BAGUIM DO MONTE', 'CBD BAIXA BANHEIRA',
       'CBD BARBOSA BOCAGE', 'CBD BARCELOS', 'CBD BOUCINHAS',
       'CBD BRAGA QT PORTAS', 'CBD BRAGA S JOSE', 'CBD BREJOS AZEITÃO',
       'CBD BUARCOS FIG FOZ', 'CBD CABECA SANTA', 'CBD CABECEIRAS BASTO',
       'CBD CALDAS RAINHA', 'CBD CALDAS TAIPAS', 'CBD CAMPO 24 AGOSTO',
       'CBD CAMPO MAIOR', 'CBD CANEDO', 'CBD CANEÇAS', 'CBD CANIDELO',
       'CBD CARNAXIDE', 'CBD CARVALHOS', 'CBD CASTELO BRANCO',
       'CBD CASTRO DAIRE', 'CBD CC ROMA', 'CBD CELEIRÓS',
       'CBD CHARNECA CAPARIC', 'CBD CHAVES', 'CBD CHOUPAL-ALVERCA',
       'CBD COIMBRA AUTOIND', 'CBD CONDEIXA', 'CBD COVA DA

# Fazer subsets da dataFrame total

> - Por loja e produto

In [37]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None) 

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

#produtos=["CAFÉ DELTA Q QALIDUS 80CAP","CAFÉ DELTA Q MYTHIQ 80CAP","CAFÉ DELTA Q QALIDUS 10CAP","CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G","CAFÉ BELLISSIMO INTENSO 200GR","CAFÉ DELTA RITUAL MU 220G","CAFÉ SOLÚVEL DELTA FRASCO 200G","CAFÉ SOLÚVEL DELTA FRASCO 100G","CEVADA SOLÚVEL DELTA FRASCO 200G","BEBIDA CEREAIS DELTA C/20%CAFE FR 200G"]
#lojas=["CNT AMADORA","CNT BRAGA","CNT CASCAIS","CNT COIMBRASHOPPING","CNT MATOSINHOS","CNT LEIRIA","CNT SANTAREM","CNT VIANA","MDL T. VEDRAS","CNT PORTIMÃO"]

In [38]:
# Definir o dataframe final apenas tendo em conta as lojas e os produtos requeridos

dfFinal = dfTudo[dfTudo['STORE_NAME'].isin(lojas) & dfTudo['DESC_ARTIGO'].isin(produtos)].copy()

In [39]:
dfFinal["STORE"].unique().tolist()

[1, 2, 3, 5, 6, 202, 209, 215, 223, 459]

# Temos
- dfFinal com as métricas todas

# Queremos
- Reposição
- Dias específicos
- Colunas de dia e semana

# Fins de Semana

- Função separar fins de semana (se quisermos em dataframes diferentes)

In [12]:
#Função para definir qual fim de semana queremos pedir
def fds(nome, df, numero):
    a = int(numero)
    b = 3*(a-1)
    nome=df[b:b+3].copy()
    return nome


> - Extrair apenas os fins de semana

In [13]:
dfFds = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])].copy()         #Sexta, Sábado e Domingo
#dfSemana = dfJuntos[dfJuntos['DATA'].dt.weekday.isin([0,1,2,3])]    #Segunda a Quinta

> - Fins de semana em causa

In [15]:
#Dias em causa (1 fds depois da páscoa)
dfFdsPiloto=dfFds.loc[(dfFds['DATA'] >= '2023-04-14') & (dfFds['DATA'] <= '2023-05-07')].copy()

#Organizar para dar para ler mais facilmente
dfFdsPiloto=dfFdsPiloto.sort_values(by=["STORE", "DESC_ARTIGO","DATA"])

> - Colunas para a **semana** e o **dia**

In [16]:
# Copiar para fazer alterações mantendo o original
dfFdsFinal = dfFdsPiloto.copy()

#Semana
dfFdsFinal['Semana'] = dfFdsFinal['DATA'].dt.isocalendar().week
dfFdsFinal['Semana'] = dfFdsFinal.groupby('Semana').ngroup() + 1

#Dia
nome_dia = dfFdsFinal['DATA'].dt.day_name().map({'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})
dfFdsFinal['Dia'] = nome_dia

# Temos:
- Uma base apenas com os fins de semana em causa com todas as métricas que estavam até agora mais a **Semana** e o **Dia** 

# Escrever

In [13]:
escrever_excel(dfFdsFinal, "2_Delta")